In [1]:
import os
import pickle
import tqdm

from glob import glob

In [2]:
dataset_path = 'dataset' # Папка с частями датасета (01, 02 и т.д.)
result_path = 'result'

In [3]:


def save_ddi(directory, result):
    book_paths = glob(os.path.join(directory, "*"))
    paths = []
    for book in book_paths:
        paths += glob(os.path.join(book, "gen_imgs", "*"))

    result_data = os.path.join(result, 'data')
    result_image = os.path.join(result, 'images')

    os.makedirs(result_data, exist_ok=True)
    os.makedirs(result_image, exist_ok=True)

    for i, path in enumerate(tqdm.tqdm(paths, desc='Collecting dataset')):
        img_path = path
        boxes_path = os.path.join(os.path.dirname(os.path.dirname(img_path)), 'gen_boxes', os.path.splitext(os.path.basename(img_path))[0] + ".pickle")
        
        with open(boxes_path, "rb") as f:
            data = pickle.load(f)

        new_img_path = os.path.join(result_image, f'DDI_{i}.png')
        os.rename(img_path, new_img_path)

        # Пишем боксы в txt файлы
        with open(os.path.join(result_data, f'DDI_{i}.txt'), 'w', encoding='utf-8') as txt:
            for info in data:
                box, text = info['box'], info['text']
                top_left, bottom_right = box[0], box[3]

                center_x, center_y = ((top_left[1] + bottom_right[1]) / 2) / new_img_path, ((top_left[0] + bottom_right[0]) / 2) / new_img_path
                width, height = (abs(bottom_right[1] - top_left[1])) / new_img_path, (abs(bottom_right[0] - top_left[0])) / new_img_path
                txt.write(f'{text} {center_x} {center_y} {width} {height}\n')

In [4]:
save_ddi(dataset_path, result_path)